# Task 4 — Comparison of PageRank Algorithms and Topic-Sensitive PageRank Implementation

## Overview

This notebook compares three PageRank variants and implements Topic-Sensitive PageRank on the citation network.

### 1. Normal (Vanilla) PageRank

**Mathematical Formulation:**
$$PR(v) = \frac{1-d}{N} + d \sum_{u \in In(v)} \frac{PR(u)}{L(u)}$$

where:
- $PR(v)$ = PageRank of page $v$
- $In(v)$ = set of pages linking to $v$
- $L(u)$ = number of outgoing links from $u$
- $N$ = total number of pages
- $d$ = damping factor (e.g., 0.85)

**Key Characteristics:**
- Models a random surfer who randomly clicks links on the web.
- With probability $d$: follows a link from the current page.
- With probability $(1-d)$: jumps to any page uniformly at random.
- Treats all pages equally (no bias).

**Usage:** Global importance ranking; web search engines; citation networks.

---

### 2. Topic-Sensitive PageRank

**Mathematical Formulation:**
$$PR_t(v) = (1-d) \cdot p_t(v) + d \sum_{u \in In(v)} \frac{PR_t(u)}{L(u)}$$

where:
- $p_t(v)$ = topic-specific teleportation vector (non-zero only for pages related to topic $t$)
- All other components same as Normal PageRank.

**Key Characteristics:**
- Random surfer is biased toward pages relevant to a specific topic.
- Teleportation jumps are biased to topic-related pages.
- With probability $d$: follows links normally.
- With probability $(1-d)$: jumps to a random topic-related page.

**Usage:** Topic-specific ranking (e.g., "security papers"); contextual search; topical authority.

---

### 3. Personalized PageRank

**Mathematical Formulation:**
$$PR_{personal}(v) = (1-d) \cdot p_{user}(v) + d \sum_{u \in In(v)} \frac{PR_{personal}(u)}{L(u)}$$

where:
- $p_{user}(v)$ = personalization vector representing user's preferences (can include browsing history, interest profile, etc.)
- Can be fully customized per user.

**Key Characteristics:**
- Generalization of Topic-Sensitive PR.
- Teleportation depends on individual user preferences.
- Each user has its own PageRank vector.
- Enables fine-grained personalization.

**Usage:** Recommendation systems; personalized search results; user-specific ranking.

---

## Comparison Table

| Aspect | Normal PageRank | Topic-Sensitive PageRank | Personalized PageRank |
|--------|-----------------|-------------------------|----------------------|
| **Teleportation** | Uniform (all pages equally) | Biased to topic-related pages | Biased to user preferences |
| **Bias** | None (global) | Topic-based | User/profile-based |
| **Use Case** | General ranking | Topic/domain-specific ranking | Individual personalization |
| **Complexity** | Low | Medium | Medium to High |
| **One-time computation?** | Yes | Compute per topic | Compute per user |

---

## Implementation: Topic-Sensitive PageRank

Below, we implement Topic-Sensitive PageRank on the citation network for five topics: **security**, **hashing**, **streaming**, **timeseries**, and **search**.

In [1]:
# Imports
import json
import os
import time
import numpy as np
import networkx as nx
from collections import defaultdict

In [2]:
# Load the subset file and build the citation graph
subset_path = os.path.join(os.getcwd(), 'dblp_subset.json')
print('Reading', subset_path)
start = time.time()
papers = []
with open(subset_path, 'r', encoding='utf-8') as f:
    for line in f:
        try:
            papers.append(json.loads(line))
        except Exception:
            continue

n = len(papers)
print(f'Read {n} papers in {time.time()-start:.2f}s')

# Build mappings
id_to_idx = {}
titles = []
refs_by_id = []
for idx, p in enumerate(papers):
    pid = p.get('id')
    id_to_idx[pid] = idx
    titles.append(p.get('title', '') or '')
    refs_by_id.append(p.get('references', []) or [])

print('Built mappings for', n, 'papers')

Reading /Users/ankushchhabra/Downloads/Data Mining Assignment2/dblp_subset.json
Read 49572 papers in 0.63s
Built mappings for 49572 papers


In [3]:
# Build directed citation graph using NetworkX
# Edge (i -> j) means paper i cites paper j
print('Building directed graph...')
start = time.time()
G = nx.DiGraph()
G.add_nodes_from(range(n))

for i, refs in enumerate(refs_by_id):
    for ref_id in refs:
        j = id_to_idx.get(ref_id)
        if j is not None:
            G.add_edge(i, j)

print(f'Graph built in {time.time()-start:.2f}s')
print(f'Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}')

Building directed graph...
Graph built in 0.36s
Nodes: 49572, Edges: 163309


In [4]:
# Define topics and find papers relevant to each topic
topics = [['security'], ['hashing'], ['streaming'], ['timeseries', 'time-series'], ['search']]

# For each topic, find papers whose title contains the topic word (case-insensitive)
topic_papers = {}
for topic in topics:
    relevant_papers = []
    for idx, title in enumerate(titles):
      for t in topic:
        if t.lower() in title.lower():
            relevant_papers.append(idx)
    topic_papers[topic[0]] = relevant_papers
    print(f'Topic "{topic}": {len(relevant_papers)} papers')
for i in range(len(topics)):
    topics[i] = topics[i][0]

Topic "security": 519 papers
Topic "hashing": 75 papers
Topic "streaming": 154 papers
Topic "timeseries": 1 papers
Topic "search": 1309 papers


In [5]:
# Function to compute Topic-Sensitive PageRank
def compute_topic_sensitive_pagerank(G, topic_nodes, alpha=0.85):
    """
    Compute Topic-Sensitive PageRank for a given set of topic-relevant nodes.
    
    Parameters:
    - G: NetworkX DiGraph
    - topic_nodes: list of node indices that are relevant to the topic
    - alpha: damping factor (default 0.85)
    
    Returns:
    - Dictionary of PageRank scores for all nodes
    """
    # Create personalization vector
    # Equal probability for topic nodes, zero for others
    personalization = {}
    n = len(G)
    
    if len(topic_nodes) == 0:
        # If no topic-specific nodes, use uniform distribution
        for node in G.nodes():
            personalization[node] = 1.0 / n
    else:
        # Distribute probability equally among topic nodes
        prob_per_topic_node = 1.0 / len(topic_nodes)
        for node in G.nodes():
            if node in topic_nodes:
                personalization[node] = prob_per_topic_node
            else:
                personalization[node] = 0.0
    
    # Compute PageRank with personalization vector
    pr = nx.pagerank(G, alpha=alpha, personalization=personalization)
    return pr

print('Function defined')

Function defined


In [6]:
# Compute Topic-Sensitive PageRank for each topic
alpha = 0.85
results = {}

for topic in topics:
    print(f'\nComputing Topic-Sensitive PageRank for "{topic}"...')
    start = time.time()
    topic_nodes = topic_papers[topic]
    pr = compute_topic_sensitive_pagerank(G, topic_nodes, alpha=alpha)
    elapsed = time.time() - start
    
    # Get top-10 papers by PageRank
    top_k = 10
    top_papers = sorted(pr.items(), key=lambda x: x[1], reverse=True)[:top_k]
    
    results[topic] = {
        'pagerank': pr,
        'top_papers': top_papers,
        'topic_nodes': topic_nodes
    }
    
    print(f'  Computed in {elapsed:.2f}s; topic-relevant nodes: {len(topic_nodes)}')


Computing Topic-Sensitive PageRank for "security"...
  Computed in 2.16s; topic-relevant nodes: 519

Computing Topic-Sensitive PageRank for "hashing"...
  Computed in 0.12s; topic-relevant nodes: 75

Computing Topic-Sensitive PageRank for "streaming"...
  Computed in 0.21s; topic-relevant nodes: 154

Computing Topic-Sensitive PageRank for "timeseries"...
  Computed in 0.10s; topic-relevant nodes: 1

Computing Topic-Sensitive PageRank for "search"...
  Computed in 0.38s; topic-relevant nodes: 1309


In [7]:
# Report results for each topic
for topic in topics:
    print('\n' + '='*90)
    print(f'TOP-10 PAPERS FOR TOPIC: {topic.upper()}')
    print('='*90)
    print(f'{"S.No.":<6} {"Title":<50} {"PageRank Score":<20} {"Citations":<10}')
    print('-'*90)
    
    top_papers = results[topic]['top_papers']
    
    for rank, (idx, pr_score) in enumerate(top_papers, start=1):
        title = titles[idx]
        # Truncate title if too long
        if len(title) > 50:
            title = title[:47] + '...'
        
        # Get in-degree (citation count)
        citation_count = G.in_degree(idx)
        
        print(f'{rank:<6} {title:<50} {pr_score:<20.8f} {citation_count:<10}')


TOP-10 PAPERS FOR TOPIC: SECURITY
S.No.  Title                                              PageRank Score       Citations 
------------------------------------------------------------------------------------------
1      Security and Privacy Challenges in Cloud Comput... 0.00895640           33        
2      Neutralization: new insights into the problem o... 0.00801610           21        
3      SecureCloud: Towards a Comprehensive Security F... 0.00731781           3         
4      Enabling Public Auditability and Data Dynamics ... 0.00675030           47        
5      Dependable and Secure Sensor Data Storage with ... 0.00674550           3         
6      Stackelberg vs. Nash in security games: interch... 0.00605933           8         
7      A lattice-based approach to mashup security        0.00565877           8         
8      Google Android: A Comprehensive Security Assess... 0.00548706           22        
9      Permissive dynamic information flow analysis       0.0051

In [8]:
# Save results to CSV files for easy reference
import csv

for topic in topics:
    csv_path = os.path.join(os.getcwd(), f'topic_sensitive_pagerank_{topic}.csv')
    top_papers = results[topic]['top_papers']
    
    with open(csv_path, 'w', newline='', encoding='utf-8') as f:
        w = csv.writer(f)
        w.writerow(['Rank', 'Paper_Index', 'Title', 'PageRank_Score', 'Citation_Count'])
        
        for rank, (idx, pr_score) in enumerate(top_papers, start=1):
            citation_count = G.in_degree(idx)
            w.writerow([rank, idx, titles[idx], f'{pr_score:.8f}', citation_count])
    
    print(f'Saved {topic} results to {csv_path}')

Saved security results to /Users/ankushchhabra/Downloads/Data Mining Assignment2/topic_sensitive_pagerank_security.csv
Saved hashing results to /Users/ankushchhabra/Downloads/Data Mining Assignment2/topic_sensitive_pagerank_hashing.csv
Saved streaming results to /Users/ankushchhabra/Downloads/Data Mining Assignment2/topic_sensitive_pagerank_streaming.csv
Saved timeseries results to /Users/ankushchhabra/Downloads/Data Mining Assignment2/topic_sensitive_pagerank_timeseries.csv
Saved search results to /Users/ankushchhabra/Downloads/Data Mining Assignment2/topic_sensitive_pagerank_search.csv


In [9]:
# Summary and Statistics
print('\n' + '='*90)
print('SUMMARY STATISTICS')
print('='*90)

for topic in topics:
    top_papers = results[topic]['top_papers']
    topic_nodes = results[topic]['topic_nodes']
    
    # Average PageRank for top-10
    avg_pr = np.mean([score for _, score in top_papers])
    
    # Average citation count for top-10
    avg_citations = np.mean([G.in_degree(idx) for idx, _ in top_papers])
    
    print(f'\nTopic: {topic.upper()}')
    print(f'  Topic-relevant papers in collection: {len(topic_nodes)}')
    print(f'  Top-10 avg PageRank score: {avg_pr:.8f}')
    print(f'  Top-10 avg citation count: {avg_citations:.2f}')


SUMMARY STATISTICS

Topic: SECURITY
  Topic-relevant papers in collection: 519
  Top-10 avg PageRank score: 0.00652840
  Top-10 avg citation count: 19.70

Topic: HASHING
  Topic-relevant papers in collection: 75
  Top-10 avg PageRank score: 0.02281168
  Top-10 avg citation count: 25.70

Topic: STREAMING
  Topic-relevant papers in collection: 154
  Top-10 avg PageRank score: 0.01108858
  Top-10 avg citation count: 10.30

Topic: TIMESERIES
  Topic-relevant papers in collection: 1
  Top-10 avg PageRank score: 0.09761608
  Top-10 avg citation count: 168.20

Topic: SEARCH
  Topic-relevant papers in collection: 1309
  Top-10 avg PageRank score: 0.00425985
  Top-10 avg citation count: 152.50


## Conclusions

Topic-Sensitive PageRank successfully ranks papers by their importance within specific topics:

1. **Bias toward topics**: By using a personalization vector biased toward topic-relevant papers, the algorithm naturally emphasizes papers that are authorities in that specific domain.

2. **Ranking changes**: Papers ranked highly in one topic may rank differently in another topic, reflecting the true structure of domain-specific citation networks.

3. **Citations vs. PageRank**: Notice that papers with higher PageRank scores in a topic often (but not always) have higher citation counts. This shows PageRank captures a richer notion of importance than simple citation count.

4. **Use cases**: Topic-Sensitive PageRank is valuable for:
   - Domain-specific literature discovery
   - Ranking papers in specialized research areas
   - Building topic-focused recommendation systems
   - Identifying influential papers within research communities

Results have been saved to individual CSV files for each topic.